In [96]:
import random
import itertools

import torch
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder,MultiLabelBinarizer

pd.set_option('display.max_rows', 500)

In [101]:
df = pd.read_csv('../data/dataset_clean.csv')
df.head()

,Source,Target,Weight
0,influenza,uncoordination,68
1,influenza,fever,68
2,influenza,pleuritic pain,68
3,influenza,snuffle,68
4,influenza,throat sore,68


In [102]:
df.isna().sum()

Source    0
Target    3
Weight    0
dtype: int64

In [103]:
df[df.Target.isna()]

,Source,Target,Weight
932,acquired immuno-deficiency syndrome,NaN,350
1861,hiv infections,NaN,350
1885,HIV,NaN,350


In [104]:
df.dropna(inplace=True)
df.drop(['Weight'], axis=1, inplace=True)

In [106]:
df.Source.nunique()

148

In [107]:
df.Target.nunique()

404

In [79]:
a = OneHotEncoder().fit(df.Target.values.reshape(-1, 1))

In [80]:
a.categories_[0][0]

"Heberden's node"

In [82]:
print(a.transform([['uncoordination']]))

  (0, 381)	1.0


In [83]:
a = np.array([0,1,2], dtype=np.uint8)
a

array([0, 1, 2], dtype=uint8)

In [84]:
df = df.groupby('Source')['Target'].apply(list).reset_index()
df.head()

,Source,Target
0,Alzheimer's disease,"[drool, agitation, nightmare, rhonchus, consci..."
1,HIV,"[fever, night sweat, spontaneous rupture of me..."
2,Pneumocystis carinii pneumonia,"[yellow sputum, cachexia, chill, decreased bod..."
3,accident cerebrovascular,"[dysarthria, asthenia, speech slurred, facial ..."
4,acquired immuno-deficiency syndrome,"[fever, night sweat, spontaneous rupture of me..."


In [85]:
dis_sym = {k:v for k,v in df[['Source', 'Target']].values}
dis_sym['HIV']

['fever',
 'night sweat',
 'spontaneous rupture of membranes',
 'cough',
 'decreased body weight',
 'chill',
 'diarrhea',
 'pleuritic pain',
 'patient non compliance',
 'tachypnea',
 'productive cough',
 'muscle hypotonia',
 'hypotonic',
 'feeling suicidal']

In [86]:
class env:
    def __init__(self, dis_sym):
        self.dis_sym = dis_sym
        self.diseases = list(dis_sym.keys())
        symptoms = [v for k,v in dis_sym.items()]
        self.symptoms = list(set(itertools.chain.from_iterable(symptoms)))
        self.curr_disease = None
        self.curr_disease_symptoms = None
        self.curr_disease_symptoms_asked = None
        self.dis_sym_enc = OneHotEncoder().fit(np.array(self.diseases + self.symptoms).reshape(-1,1))
        self.sym_enc = MultiLabelBinarizer().fit(np.array(self.symptoms).reshape(-1, 1))
        
    def reset():
        self.curr_disease = random.choice(diseases)
        self.curr_disease_symptoms = self.dis_sym[self.curr_disease]
        first_symptom = random.choice(self.curr_disease_symptoms)
        self.curr_disease_symptoms_asked.append(first_symptom)
        return self.curr_disease_symptoms_asked
    
    def step(action):
        action = self.dis_sym_enc.inverse_transform(action)
        if action == self.curr_disease:
            done = True
            reward = 44
            observation_ = self.curr_disease_symptoms_asked
        elif action in self.diseases:
            done = True
            reward = -22
            observation_ = self.curr_disease_symptoms_asked
        elif action in self.curr_disease_symptoms:
            done = False
            reward = -1
            self.curr_disease_symptoms_asked.append(action)
            observation_ = self.curr_disease_symptoms_asked
        else:
            done = False
            reward = -1
            observation_ = self.curr_disease_symptoms_asked
            
        observation_ = self.sym_enc.transform([self.curr_disease_symptoms_asked])
        info = ''
            
        return observation_, reward, done, info